In [14]:
import pandas as pd
import json
import requests
from pprint import pprint

In [2]:
url = "https://public.opendatasoft.com/api/records/1.0/search/?dataset=global-shark-attack&q=&facet=date&facet=type&facet=country&facet=area&facet=activity&facet=sex&facet=age&facet=fatal_y_n&facet=time&facet=species"

response = requests.get(url)    # Will get a HTTP Status code (response object)

response_json = response.json()

In [18]:
# Just want to pull records 
records = response_json['records']

# Look at first record
pprint(records[1])

{'datasetid': 'global-shark-attack',
 'fields': {'activity': 'Surfing',
            'area': 'Noirth Carolina',
            'case_number': '2022.07.19.a',
            'case_number0': '2022.07.19.a',
            'case_number1': '2022.07.19.a',
            'country': 'USA',
            'date': '2022-07-19',
            'fatal_y_n': 'N',
            'href': 'http://sharkattackfile.net/spreadsheets/pdf_directory/2022.07.19.a-Vreuis.pdf',
            'href_formula': 'http://sharkattackfile.net/spreadsheets/pdf_directory/2022.07.19.a-Vreuis.pdf',
            'injury': 'Lacerations to lower leg',
            'investigator_or_source': 'C. Creswell. GSAF',
            'location': 'Topsail Beach, Pender County',
            'name': 'Erika Vreuls',
            'original_order': '6787',
            'pdf': '2022.07.19.a-Vreuis.pdf',
            'sex': 'F',
            'time': 'Early  morning',
            'type': 'Unprovoked',
            'year': '2022'},
 'record_timestamp': '2023-01-15T11:00:04.38

In [19]:
# Need to make a list of the field dictionaries
records_fields = [x["fields"] for x in records]

# Check list to make sure correct info was collected
records_fields[0:1]

[{'activity': 'Surfing',
  'location': 'Centerville Beach',
  'sex': 'M',
  'year': '2022',
  'injury': 'Thigh injured',
  'case_number': '2022.010.02',
  'time': '15h30',
  'name': 'Jared Trainor',
  'investigator_or_source': 'Outsider, 10/3/2022',
  'age': '31',
  'species': "White shark, 13'",
  'date': '2022-10-02',
  'type': 'Unprovoked',
  'area': 'California',
  'country': 'USA',
  'fatal_y_n': 'N'},
 {'activity': 'Surfing',
  'location': 'Topsail Beach, Pender County',
  'sex': 'F',
  'year': '2022',
  'injury': 'Lacerations to lower leg',
  'case_number': '2022.07.19.a',
  'href_formula': 'http://sharkattackfile.net/spreadsheets/pdf_directory/2022.07.19.a-Vreuis.pdf',
  'original_order': '6787',
  'time': 'Early  morning',
  'case_number1': '2022.07.19.a',
  'name': 'Erika Vreuls',
  'investigator_or_source': 'C. Creswell. GSAF',
  'date': '2022-07-19',
  'type': 'Unprovoked',
  'area': 'Noirth Carolina',
  'country': 'USA',
  'fatal_y_n': 'N',
  'href': 'http://sharkattackfil

In [20]:
# Make a dataframe
df = pd.DataFrame(records_fields)
df.head(5)

,activity,location,sex,year,injury,case_number,time,name,investigator_or_source,age,...,type,area,country,fatal_y_n,href_formula,original_order,case_number1,href,pdf,case_number0
0,Surfing,Centerville Beach,M,2022,Thigh injured,2022.010.02,15h30,Jared Trainor,"Outsider, 10/3/2022",31,...,Unprovoked,California,USA,N,NaN,NaN,NaN,NaN,NaN,NaN
1,Surfing,"Topsail Beach, Pender County",F,2022,Lacerations to lower leg,2022.07.19.a,Early morning,Erika Vreuls,C. Creswell. GSAF,NaN,...,Unprovoked,Noirth Carolina,USA,N,http://sharkattackfile.net/spreadsheets/pdf_di...,6787,2022.07.19.a,http://sharkattackfile.net/spreadsheets/pdf_di...,2022.07.19.a-Vreuis.pdf,2022.07.19.a
2,Swimming,"Sawyer Key , Monroe County",F,2022,Laceration to leg,2022.06.29.b,20h00,Lindsay Rebecca Bruns,Miami Herald/ 7/12022,35.0,...,Unprovoked,Florida,USA,N,http://sharkattackfile.net/spreadsheets/pdf_di...,6772,2022.06.29.b,http://sharkattackfile.net/spreadsheets/pdf_di...,2022.06.29.b-Bruns.pdf,2022.06.29.b
3,Swimming,Oyster Stacks near Exmouth12h15,F,2022,Minor injury,2022.06.01,12h15,NaN,"Perth Now, 6//2/2022",NaN,...,Unprovoked,Western Australia,AUSTRALIA,N,http://sharkattackfile.net/spreadsheets/pdf_di...,6765,2022.06.01,http://sharkattackfile.net/spreadsheets/pdf_di...,2022.06.01-Exmouth.pdf,2022.06.01
4,Swimming,Pororari River,F,2021,Laceration to big toe,2021.12.25,NaN,Cordelia Scott,"New Zealand Herald, 12/31/2021",9.0,...,Unprovoked,South Island,New Zealand,N,http://sharkattackfile.net/spreadsheets/pdf_di...,6726,2021.12.25,http://sharkattackfile.net/spreadsheets/pdf_di...,2021.12.25-Scott.pdf,2021.12.25


In [22]:
# Make a cut pd with the fields: ["activity", "location", "sex", "year", "injury", "case_number", "time", "age", "type", "area", "country", "fatal_y_n", "species"]
df_cut = df.loc[:, ["activity", "location", "sex", "year", "injury", "case_number", "time", "age", "type", "area", "country", "fatal_y_n", "species"]]
df_cut.head(5)

,activity,location,sex,year,injury,case_number,time,age,type,area,country,fatal_y_n,species
0,Surfing,Centerville Beach,M,2022,Thigh injured,2022.010.02,15h30,31,Unprovoked,California,USA,N,"White shark, 13'"
1,Surfing,"Topsail Beach, Pender County",F,2022,Lacerations to lower leg,2022.07.19.a,Early morning,NaN,Unprovoked,Noirth Carolina,USA,N,NaN
2,Swimming,"Sawyer Key , Monroe County",F,2022,Laceration to leg,2022.06.29.b,20h00,35.0,Unprovoked,Florida,USA,N,NaN
3,Swimming,Oyster Stacks near Exmouth12h15,F,2022,Minor injury,2022.06.01,12h15,NaN,Unprovoked,Western Australia,AUSTRALIA,N,NaN
4,Swimming,Pororari River,F,2021,Laceration to big toe,2021.12.25,NaN,9.0,Unprovoked,South Island,New Zealand,N,NaN
